# Projeto 01

## Nome: Rafael Almada

### Com o aumento do IDH, o consumo de álcool e cigarro tende a cair?

O IDH (Índice de Desenvolvimento Humano) é a forma como medimos o desenvolvimento humano em cada país, logo ele é calculado a partir de dados como a educação, expectativa de vida e o PIB per capita. Porém sabemos que além disso, muitas variáveis possuem uma correlação com o IDH, sendo assim impactam indiretamente em seu valor.

O objetivo deste projeto é, por meio de uma análise descritiva de diversos Data Frames, verificar se com o aumento do IDH, o consumo de álcool e cigarro tende a diminuir.

É importante ressaltar que as bases de dados usadas foram todas retiradas do site: https://www.gapminder.org/

## Introdução

Sabe-se que o consumo de álcool e cigarro é muito utilizado, sendo a dependencia química dos mesmos muito associada a tempos difíceis. Conhecemos histórias como a de Mário de Andrade, poeta e escritor brasileiro da primeira geração do modernismo no país. Ao ver o seu amado país sob uma ditadura (Estado Novo de Getúlio Vargas, pós-1937), aquilo que ele mais repugnava, sente como se tivesse falhado em sua missão. Um movimento que falava tanto de liberdade em tantos sentidos e em se assumir uma nacionalidade própria, o tal do "Brasil brasileiro". Assim, Mário passou a beber, transformou a bebida em sua fuga da  realidade, tristeza e melancolia recaíam sobre o mesmo. Até que em 1945, já alcoólatra há alguns anos, ele morre devido a um infarto do miocárdio, o que hoje sabemos que isso só aconteceu devido ao consumo excessivo de álcool.

Já cigarros tem o mesmo peso negativo a uma pessoa, sendo no geral uma fuga ao stress. Seus efeitos são de prazer e satisfação. Sabemos também que o aumento do uso de cigarros se deve, em parte, a urbanização. Também sabemos que pode ser muito prejudicial a saúde se usado com frequencia e manter o hábito por anos.

Dado isso, podemos associar o IDH a um sentimento de satisfação com o próprio país também, sendo assim um país com maior IDH possui "menos problemas" e por isso seria uma razão para, no decorrer do tempo, haver uma diminuição no uso dessas substâncias.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

% matplotlib inline

In [2]:
alcool=pd.read_csv('alcohol_consumption.csv')
GDPcapita=pd.read_csv('gdppercapita_us_inflation_adjusted.csv')
IDH=pd.read_csv('hdi_human_development_index.csv')
life=pd.read_csv('life_expectancy_years.csv')
cigarro=pd.read_csv('smoking_adults_percent_of_population_over_age_15.csv') #singaro da cãnsir

### Join dos DataFrames!

Irei usar o ano de 2005 como referência, pois é o ano em comum entre os 3 dataframes!

In [3]:
GDP_2005=GDPcapita[['geo','2005']].copy()
IDH_2005=IDH[['geo','2005']].copy()
alc_2005=alcool[['geo','2005']].copy()
cig_2005=cigarro[['geo','2005']].copy()

In [4]:
IDH_idx=IDH_2005.set_index('geo')
alc_idx=alc_2005.set_index('geo')
cig_idx=cig_2005.set_index('geo')
GDP_idx=GDP_2005.set_index('geo')

In [5]:
IDH_alc=IDH_idx.join(alc_idx, how='inner', lsuffix='_idh', rsuffix='_alc')
IDH_cig=IDH_alc.join(cig_idx, how='inner', rsuffix='_cig')
df=IDH_cig.join(GDP_idx, how='inner', lsuffix='_cig', rsuffix='_GDP')
df.head() #dataframe pronto para a analise!

,2005_idh,2005_alc,2005_cig,2005_GDP
geo,,,,
Albania,0.696,6.68,22.4,3190.0
Algeria,0.686,0.96,15.2,4270.0
Andorra,NaN,15.50,32.9,48800.0
Argentina,0.782,10.00,30.0,8520.0
Armenia,0.692,11.30,29.6,2570.0
